In [12]:
import yfinance as yf
import datetime as dt
import pandas as pd
from IPython.display import display_json

In [13]:
start_date = '1970-01-01'
end_date = dt.datetime.today()

In [4]:
stock = yf.Ticker("O")
df = pd.DataFrame(columns=["div yield", "Trailing annual div yield", "exDiv", "payout ratio"])
# df.loc["O"] = "24-12-22", "3.5%"
# df.loc["AMZN"] = "23-11-22", "2%"
df

,div yield,Trailing annual div yield,exDiv,payout ratio


In [163]:
stock = yf.Ticker("ABR")
tmp = stock.dividends
# list(data.columns)
# data.columns
df = pd.DataFrame(index=tmp.index)
df["Dividends"] = tmp
# df["Growth rate"] = ((df["Dividends"]/df["Dividends"].shift(1)) - 1) * 100
# df['Year'] = df.index.year
df.index = df.index.year
df = df.groupby(df.index).mean()
# df.columns
df["Growth year"] = ((df["Dividends"]/df["Dividends"].shift(1)) - 1) * 100
df["Growth rate"] = df["Growth year"].expanding().mean()
df["Years grow"] = df.index - df.first_valid_index()
df

,Dividends,Growth year,Growth rate,Years grow
Date,,,,
2004,0.3900,NaN,NaN,0
2005,0.5600,43.589744,NaN,1
2006,0.6425,14.732143,NaN,2
2007,0.6150,-4.280156,NaN,3
2008,0.5250,-14.634146,NaN,4
2012,0.0950,-81.904762,-8.499435,8
2013,0.1250,31.578947,-10.901595,9
2014,0.1300,4.000000,-13.048023,10
2015,0.1450,11.538462,-9.884300,11


In [154]:
tickers = ["O", "T", "MMM", "IBM", "PSTL", "ENB", "BCE", "CSCO", "MRK", "STAG", "INPFX", "TIP", "AAPL", "MSFT", "JPM", "KO", "V", "COST", "JNJ", "VICI", "MCD", "ABBV", "ABR"]
# tickers = ["O", "T", "MMM"]

In [155]:
df = pd.DataFrame(columns=["div yield", "Trailing annual div yield", "exDiv", "payout ratio", "5 year yield average", "Growth rate", "Years of grow"])

In [156]:
tickers = yf.Tickers(tickers)
for ticker in tickers.tickers:
    stock = yf.Ticker(ticker)
    data = stock.info

    div_yield = data["dividendYield"]  # * 100
    div_ex = pd.to_datetime(data["exDividendDate"], unit='s')  # Ex date
    payout = data["payoutRatio"] # * 100
    trail_div_yield = data["trailingAnnualDividendYield"]  # * 100
    five_y_yield_avg = data["fiveYearAvgDividendYield"]

    tmp = stock.dividends
    div_df = pd.DataFrame(index=tmp.index)
    div_df["Dividends"] = tmp
    div_df.index = div_df.index.year
    div_df = div_df.groupby(div_df.index).mean()
    div_df["Growth year"] = ((div_df["Dividends"]/div_df["Dividends"].shift(1)) - 1) * 100
    div_df["Growth rate"] = div_df["Growth year"].expanding().mean()
    div_df["Years grow"] = div_df.index - div_df.first_valid_index()
    growth_rate = float(div_df.tail(1)["Growth rate"])
    growth_year = int(div_df.tail(1)["Years grow"])  # Years of growth

    df.loc[ticker] = div_yield, trail_div_yield, div_ex, payout, five_y_yield_avg, growth_rate, growth_year
df["payout ratio"] *= 100
df["div yield"] *= 100
df["Trailing annual div yield"] *= 100

df.sort_index()

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,Growth rate,Years of grow
AAPL,0.65,0.625174,2022-11-04,14.729999,0.98,468.068384,35.0
ABBV,4.02,3.826323,2023-01-12,73.730004,4.36,14.391911,10.0
ABR,10.91,10.497614,2022-11-17,78.950000,9.17,7.509414,18.0
BCE,5.86,7.750533,2022-12-14,117.640000,5.52,149.974023,34.0
COST,0.72,0.700381,2023-02-02,26.380000,0.82,74.265842,18.0
CSCO,3.14,3.123707,2023-01-04,54.510000,2.93,18.505391,12.0
ENB,6.43,8.400983,2023-02-14,126.480010,6.49,10.221437,32.0
IBM,4.91,4.901451,2022-11-09,337.950020,4.82,18.628999,60.0
INPFX,NaN,NaN,NaT,NaN,NaN,1.337350,10.0
JNJ,2.68,2.634851,2023-02-17,66.120005,2.59,13.630227,60.0


In [161]:
df.sort_values("div yield", ascending=False)

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,Growth rate,Years of grow
ABR,10.91,10.497614,2022-11-17,78.950000,9.17,7.509414,18.0
ENB,6.43,8.400983,2023-02-14,126.480010,6.49,10.221437,32.0
PSTL,6.14,6.045752,2022-11-04,653.570000,NaN,37.082617,3.0
BCE,5.86,7.750533,2022-12-14,117.640000,5.52,149.974023,34.0
T,5.44,0.000000,2023-01-09,NaN,7.00,2.665191,39.0
MMM,5.28,5.248789,2022-11-17,58.550000,3.38,9.025397,60.0
IBM,4.91,4.901451,2022-11-09,337.950020,4.82,18.628999,60.0
VICI,4.59,4.326074,2022-12-21,140.000000,NaN,10.763204,4.0
O,4.39,4.344564,2023-01-31,298.280000,4.31,4.640202,28.0
STAG,4.21,4.104729,2023-01-30,108.770000,4.67,-4.370850,11.0


,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average
AAPL,0.65,0.625174,2022-11-04,14.729999,0.98
ABBV,4.02,3.826323,2023-01-12,73.730004,4.36
ABR,11.14,10.497614,2022-11-17,78.950000,9.17
BCE,5.94,7.750533,2022-12-14,117.640000,5.52
COST,0.72,0.700381,2023-02-02,26.380000,0.82
CSCO,3.14,3.123707,2023-01-04,54.510000,2.93
ENB,6.43,8.400983,2023-02-14,126.480010,6.49
IBM,4.91,4.901451,2022-11-09,337.950020,4.82
INPFX,NaN,NaN,NaT,NaN,NaN
JNJ,2.68,2.634851,2023-02-17,66.120005,2.59
